In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk 
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
pd.set_option('display.max_colwidth', 100)

In [ ]:
df = pd.read_csv('/kaggle/input/sinhala-unicode-hate-speech/sinhala-hate-speech-dataset.csv', encoding='utf8')

Let's explore the dataset :)

In [ ]:
df.shape

Dataset contains 6345 rows and 3 columns.

In [ ]:
df.columns

In [ ]:
df.head()

Let's, check missing values.

In [ ]:
count = df.isna().sum()
precentange = df.isna().mean().round(4) * 100

dfMissing = pd.DataFrame({'count': count, 'precentange': precentange})
dfMissing.sort_values('count', ascending=False)

Good news, dataset doesn't contain any missing values.

Let's see how data has been distributed.

In [ ]:
df.groupby(['label'])['label'].describe()[['count']]

In [ ]:
sns.countplot(x='label', data=df).set_title('Hate Speech Distribution')
df.label.value_counts(normalize=True) * 100

54% of comments are labelled as hate speeches and 45% of comments are labelled as non hate speeches. Okay now we have some idea about comments. let's calcualte length of comments.

In [ ]:
df['length'] = df['comment'].apply(len)
df.head()

In [ ]:
df.hist(column='length',by='label',figsize=(15,6), bins=150)

According to the histograms, cannot see significant different between length of hate and normal comments.

In [ ]:
df[df['label'] == 1]['length'].describe()

It's time to preprocess the data, we can see there are some unwanted data in the comments such as numbers, punctuations which do not add significant meaning to the data. Therefore let's write functions to remove numbers and punctuations.


We don' need id colum therefore let's remove that.

In [ ]:
del df['id']

In [ ]:
exclude = set(",.:;'\"-?!/´`%")
def removePunctuation(txt):
    return ''.join([(c if c not in exclude else " ") for c in txt])
    
def removeNumbers(txt):
    return ''.join(c for c in txt if not c.isnumeric())

Let's remove numbers first and assign cleaned data to new column.


In [ ]:
df['cleaned'] = df['comment'].apply(lambda x: removePunctuation(x))
df.head()

Okay, we can see now punctuations have been removed from the comments. As a second step, let's remove numbers and assign cleaned data to new column.


In [ ]:
df['cleaned'] = df['cleaned'].apply(lambda x: removeNumbers(x))
df.head()

Okay, I think that's the only cleaning we need. Therefore let's tokenize the comments. Let's check most common 10 words.


In [ ]:
from collections import Counter

plt.rc('font', family='Lohit Devanagari')

results = Counter()
df.cleaned.str.split().apply(results.update)

most = results.most_common()
print(most[:10])

Let's see most common 20 words of both hate and normal comments.

In [ ]:
words = [word for i in df[df['label'] == 1]['cleaned'].str.split() for word in i]

counter = Counter(words)
most = counter.most_common(20)
    
most

In [ ]:
words = [word for i in df[df['label'] == 0]['cleaned'].str.split() for word in i]

counter = Counter(words)
most = counter.most_common(20)
    
most